In [21]:
import os, math, random, argparse, ast, copy
from collections import Counter, defaultdict
from itertools import chain, combinations
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import os
import math
import random
import argparse
from collections import Counter, defaultdict
from itertools import chain, combinations
import ast
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression
from utils.Spambase.split_data import split_data_equal
from utils.aggregate_functions import aggregate_lr_models
from utils.evaluate_coalitions_new import evaluate_coalitions2
from utils.aggregate_functions import FederatedForest
from utils.DecisionTree import DecisionTree

from utils.nash1 import find_nash_equilibria_v2


import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", message="X does not have valid feature names")

import os
import math
import random
import argparse
import ast
import copy

import numpy as np
import pandas as pd
from pathlib import Path

from collections import Counter, defaultdict
from itertools import chain, combinations
from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

from typing import Tuple, Sequence, Union


### Local Imports

In [22]:
from utils.Spambase.split_data import split_data_equal
from utils.aggregate_functions import FederatedForest
from utils.DecisionTree import DecisionTree
from utils.nash1 import find_nash_equilibria_v2
from utils.evaluate_coalitions_new import evaluate_coalitions2

In [23]:
from utils.Nash_lottery import find_nash_equilibria_lottery

# HuGaDB Dataset 

In [24]:
import numpy as np
import joblib

# 1) load the pipeline
pipeline = joblib.load('/Users/abbaszal/Documents/global_acc_pipeline_fedfor.joblib')


In [25]:
# File patterns.
train_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/train_{i:02d}.csv"
test_files_pattern = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/test_{i:02d}.csv"

# Concatenate all training files.
df_train_global = pd.concat([
    pd.read_csv(train_files_pattern.format(i=i)) for i in range(1, 11)
]).dropna()

# Concatenate all testing files.
df_test_global = pd.concat([
    pd.read_csv(test_files_pattern.format(i=i)) for i in range(1, 11)
]).dropna()

# Split features and labels.
X_train_global = df_train_global.drop('act', axis=1)
y_train_global = df_train_global['act']

X_test_global = df_test_global.drop('act', axis=1)
y_test_global = df_test_global['act']

# Encode labels.
label_encoder = LabelEncoder()
y_train_global = label_encoder.fit_transform(y_train_global)
y_test_global = label_encoder.transform(y_test_global)

# Scale features.
scaler_global = StandardScaler()
X_train_global_scaled = scaler_global.fit_transform(X_train_global)
X_test_global_scaled  = scaler_global.transform(X_test_global)

In [27]:
# Parameters
eps               = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials          = 100
base_random_seed  = 42
max_depths        = [100]
approach          = 'fedfor'

# Saving directory
save_dir = "/Users/abbaszal/Documents/thimasfit11/hugadb_fedfor_all_test"
os.makedirs(save_dir, exist_ok=True)



train_csv_path = "/Users/abbaszal/Documents/Thesis_Project_Spambase/data/metadata/new_runs/train.csv"
df_full_train = pd.read_csv(train_csv_path).dropna(subset=['act']).reset_index(drop=True)

for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")
          
    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]
 

    for max_depth in max_depths:
        print(f"\n max_depth = {max_depth}")

        # reset counters for this config
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials + 1):
            
            rand_component = random.randint(0, 500)
            trial_seed = base_random_seed + trial + int(1000 * max_depth) + 2 * rand_component



            X_test_full_scaled = X_test_global_scaled.copy()
            y_test_full        = y_test_global.copy()

            # Create a stratified subsample of the test set to speed up the runtime.
            subsample_size = 950  
            X_test_glob, _, y_test_glob, _ = train_test_split(
                X_test_full_scaled, y_test_full,
                train_size=subsample_size,
                random_state=trial_seed,
                stratify=y_test_full
            )



            df_trial, _ = train_test_split(
                df_full_train,
                train_size=10000,
                random_state=trial_seed,
                stratify=df_full_train['act']
            )
   
            df_remaining = df_trial.copy()


            client_partitions = []

            sample_size = 5000 // n_clients
            for client_idx in range(n_clients):
                if len(df_remaining) < sample_size:
                    raise ValueError("err")

                if len(df_remaining) == sample_size:
                    df_client = df_remaining.copy()
                    df_remaining = df_remaining.iloc[0:0] 

                else:
                    df_client, df_remaining = train_test_split(
                        df_remaining,
                        train_size=sample_size,
                        random_state=trial_seed,
                        stratify=df_remaining['act']
                    )
                df_client = df_client.reset_index(drop=True)
                client_partitions.append(df_client)

            # train each client
            client_models = []
            client_global_accuracies   = {}

            for client_idx, df_client in enumerate(client_partitions):

                X_client = df_client.drop(columns=['act'])
                y_client = df_client['act']

                client_scaler = StandardScaler()
                X_client_scaled = client_scaler.fit_transform(X_client)
                y_client_encoded = label_encoder.transform(y_client)
                # fit model
                model = DecisionTree(
                    max_depth=max_depth,
                    random_state=trial_seed
                )
                model.fit(X_client_scaled, y_client_encoded)
                client_models.append(model)

                # record its global-test accuracy
                y_pred_global = model.predict(X_test_glob)
                acc_global = accuracy_score(y_test_glob, y_pred_global)
                client_global_accuracies[client_idx] = acc_global

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_global_accuracies,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test_glob,
                y_test=y_test_glob,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals      = np.array(list(client_global_accuracies.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)
            X_new    = np.array([[mu_full, sig_full]])   # shape (1,2)
            payoff_f    = pipeline.predict(X_new)[0]
            print(payoff_f)
            #payoff_f  = payoff_100(mu_full, sig_full)
            #print(payoff_f)

            has_incentive = any(
                acc > payoff_f + eps
                for acc in client_global_accuracies.values()
            )
            if not has_incentive:
                lottery_count += 1


        static_count = sum(counts_static.values())

        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_depth,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                 f"_nclients_{n_clients}"
                 f"_maxiter_{max_depth}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f"saved {fname}")



> n_clients = 10

 max_depth = 100
2.2294588012820844
2.422219767390399
2.0739320661023983
2.151024936982922
2.3243737865707716
2.0534812386829895
2.136991971041554
2.1343798173282327
2.0572337391613806
2.2639188366458596
2.086344677234131
2.2760052855938184
2.1307439507487604
2.540209826559402
2.109760567704733
2.3623396668298415
2.2629230557380535
2.3057333586622315
2.38627571634067
2.2651277631676967
2.306042749499422
2.012795520376433
2.292252116720241
2.468517127580828
2.253768080187318
2.2046855725228873
2.192179733877243
2.283723103025649
2.2843092870955397
2.4011572032437405
2.2067613278484757
2.0293635347093897
2.40166265570672
2.229671687538805
2.0421713772758956
2.6309557067151577
2.4767125476904406
2.1706543348011693
2.078153827144023
2.19737217898283
2.1307439507487604
2.215367159549375
2.463222365258397
2.100510039178676
2.305366177634082
2.305393450204116
2.369687478394817
2.3489290230529036
2.372005627958676
2.1382192820605708
2.32082480949169
2.259949675550714
2.51187

# Spambase Dataset

In [7]:
file_path = '/Users/abbaszal/Documents/Thesis_Project_Spambase/data/spambase.data'  # Adjust the path as needed
df = pd.read_csv(file_path, header=None)

In [8]:
import numpy as np
import joblib

# 1) load the pipeline
pipeline = joblib.load('/Users/abbaszal/Documents/global_acc_pipeline_fedfor.joblib')


In [9]:
def train_models_fedfor(partitions, X_test, y_test, max_depth):
    client_models = []
    client_global_accuracies = {}
    
    for i, (X_i, y_i) in enumerate(partitions):
        local_scaler = StandardScaler()
        model = DecisionTreeClassifier(max_depth=max_depth, random_state=np.random.randint(0, 100000))
        model.fit(local_scaler.fit_transform(X_i), y_i)
        client_models.append(model)
        y_pred = model.predict(X_test)
        client_global_accuracies[i] = np.mean(y_pred == y_test)
    return client_models, client_global_accuracies

In [10]:
# Parameters
eps     = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials    = 100
base_seed   = 42
max_depths   = [ 100]
approach    = 'fedfor'


save_dir = (
"/Users/abbaszal/Documents/thimasfit11/spambase_fedfor_all_joblib"
)
os.makedirs(save_dir, exist_ok=True)

all_results = [] 


for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")
          
    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]
 

    for max_depth in max_depths:
        print(f"\n max_depth = {max_depth}")
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials+1):
            rc = random.randint(0, 500)
            trial_seed = base_seed + (trial-1) + 1000*max_depth + 2*rc
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            X = df.iloc[:, :-1].to_numpy()
            y = df.iloc[:, -1].to_numpy()

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial_seed)

            scaler = StandardScaler()

            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            partitions = split_data_equal(
                X_train, y_train,
                n_clients=n_clients,
                shuffle=True,
                random_seed=trial_seed
            )
            client_models, client_accs = train_models_fedfor(
                partitions=partitions,
                X_test=X_test,
                y_test=y_test,
                max_depth=max_depth
            )

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test,
                y_test=y_test,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals = np.array(list(client_accs.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)

            X_new    = np.array([[mu_full, sig_full]])   # shape (1,2)
            payoff_f    = pipeline.predict(X_new)[0]
            print(payoff_f)
            #payoff_f  = payoff_100(mu_full, sig_full)
            #print(payoff_f)

            has_incentive = any(
                acc > payoff_f 
                for acc in client_accs.values()
            )
            if not has_incentive:
                lottery_count += 1

        # aggregate counts into single numbers
        static_count = sum(counts_static.values())


        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_depth,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                    f"_nclients_{n_clients}"
                    f"_maxiter_{max_depth}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f" saved {fname}")


> n_clients = 10

 max_depth = 100
0.8901809978017551
0.9076381025305283
0.9211812625652237
0.9014416405182212
0.904582505865667
0.9330766771047528
0.9105680777069347
0.8992214188678955
0.9184459857854721
0.8856656088697513
0.8695167910552385
0.901311381569936
0.8941447901270322
0.8889093479448134
0.906137586672569
0.9031393968171115
0.9081175535093533
0.9002483214333485
0.9024299550321452
0.91610048524125
0.9133031466355844
0.9046591383378187
0.9110711224859698
0.8925886154824083
0.9001442597093624
0.92251973105527
0.8934196415384568
0.9289484537608117
0.8937313580088739
0.8927627918009534
0.9018664334565473
0.8902465005866047
0.9000800506951651
0.885378977174214
0.9030995205724452
0.9032970915463803
0.8958632296707446
0.8917199117196554
0.9051435253592943
0.9257259873690449
0.907612300280151
0.9513715954050492
0.9108436002154138
0.9228311525566787
0.8905227060614815
0.8821482547440549
0.8902905079738029
0.9114686151290252
0.8838289002588047
0.9012027642899046
0.8920759575201691
0.89

### Mushroom

In [16]:
from data_loading import load_mushroom

In [17]:
(X_train, y_train, X_test, y_test), num_classes, n_num_feats, n_cat_feats, n_cat_vals = load_mushroom()
print("Train shape:", X_train[0].shape, X_train[1].shape) 
print("Test shape:", X_test[0].shape, X_test[1].shape)
print("Number of classes:", num_classes)

Train shape: (7311, 0) (7311, 22)
Test shape: (813, 0) (813, 22)
Number of classes: 2


In [18]:
(X_train, y_train, X_test, y_test), num_classes, n_num_feats, n_cat_feats, n_cat_vals = load_mushroom()


X_train_num, X_train_cat = X_train
X_test_num, X_test_cat = X_test


X_train_combined = np.hstack([X_train_num, X_train_cat])
X_test_combined  = np.hstack([X_test_num,  X_test_cat])


preprocessor = ColumnTransformer([
    ('num', StandardScaler(), list(range(n_num_feats))),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), 
     list(range(n_num_feats, n_num_feats + n_cat_feats)))
])
X_train_global_scaled = preprocessor.fit_transform(X_train_combined)
X_test_global_scaled  = preprocessor.transform(X_test_combined)


y_train_global = y_train
y_test_global  = y_test

In [19]:
import numpy as np
import joblib

# 1) load the pipeline
pipeline = joblib.load('/Users/abbaszal/Documents/global_acc_pipeline_fedfor.joblib')


In [20]:
# Prepare training DataFrame
t_feat = X_train_global_scaled.shape[1] - n_num_feats
feat_names = [f"num{i+1}" for i in range(n_num_feats)] + \
                [f"cat{i+1}" for i in range(t_feat)]
df_train = pd.DataFrame(X_train_global_scaled, columns=feat_names)
df_train["target"] = y_train_global

eps               = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials          = 100
base_random_seed  = 42
max_iters         = [100]
approach          = 'fedfor'
model_type = 'des'

# Saving directory
save_dir = "/Users/abbaszal/Documents/thimasfit11/mushroom_fedfor_all_joblib"
os.makedirs(save_dir, exist_ok=True)

for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]


    for max_iter in max_iters:
        print(f"  max_iter = {max_iter}")

        # reset counters for this config
        counts_static = Counter()
        lottery_count = 0

        for trial in range(1, n_trials + 1):
            print(trial)

            rand_component = random.randint(0, 500)
            trial_seed = base_random_seed + trial + int(1000 * max_iter) + 2 * rand_component

            X_test_full_scaled = X_test_global_scaled.copy()
            y_test_full        = y_test_global.copy()

            subsample_fraction = 0.80

            X_test_glob, _, y_test_glob, _ = train_test_split(
                X_test_full_scaled, y_test_full,
                train_size=subsample_fraction,
                random_state=trial_seed,
                stratify=y_test_full
            )



            #from sklearn.model_selection import StratifiedKFold
            #skf = StratifiedKFold(n_splits=n_clients, shuffle=True, random_state=trial_seed)
            #parts = []
            #for _, client_idx in skf.split(df_train.drop(columns="target"), df_train["target"]):
                #df_c = df_train.iloc[client_idx].reset_index(drop=True)
                #parts.append(df_c)


            df_rem      = df_train.sample(frac=1, random_state=trial_seed).reset_index(drop=True)
            sample_size = len(df_train) // n_clients
            parts = []
            for idx in range(n_clients):
                if idx == n_clients - 1:
                    df_c = df_rem.copy()
                else:
                    sss = StratifiedShuffleSplit(n_splits=1, train_size=sample_size,
                                                    random_state=trial_seed)
                    tr_idx, _ = next(sss.split(df_rem, df_rem["target"]))
                    df_c      = df_rem.iloc[tr_idx]
                    df_rem    = df_rem.drop(df_c.index).reset_index(drop=True)
                parts.append(df_c.reset_index(drop=True))

            client_models     = []
            client_accs = {}
            for idx, df_c in enumerate(parts):
                X_c = df_c.drop(columns="target").to_numpy()
                y_c = df_c["target"].to_numpy()
                X_std = StandardScaler().fit_transform(X_c)

                if model_type.lower() == "logistic":
                    model = LogisticRegression(random_state=trial_seed,
                                                max_iter=max_iter)
                else:
                    model = DecisionTree(max_depth=max_iter,
                                                    random_state=trial_seed)

                model.fit(X_std, y_c)
                client_models.append(model)
                
    
                acc_global = accuracy_score(y_test_glob,
                                            model.predict(X_test_glob))
                client_accs[idx] = acc_global
            

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test_glob,
                y_test=y_test_glob,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals      = np.array(list(client_accs.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)
            
            # 3) package into a 2-column array and predict
            X_new    = np.array([[mu_full, sig_full]])   # shape (1,2)
            payoff_f    = pipeline.predict(X_new)[0]
            print(payoff_f)


            has_incentive = any(
                acc > payoff_f + eps
                for acc in client_accs.values()
            )
            if not has_incentive:
                lottery_count += 1


        static_count = sum(counts_static.values())

        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_iter,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count,
        }])
        fname = (f"Nash_Counts_{approach}"
                 f"_nclients_{n_clients}"
                 f"_maxiter_{max_iter}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f"saved {fname}")



> n_clients = 10
  max_iter = 100
1
0.9240386737209786
2
0.9252672653217832
3
0.9053930752960212
4
0.9136150030836455
5
0.9146823388764801
6
0.9214769482407944
7
0.9152411283701805
8
0.9173759781476982
9
0.9211923016037922
10
0.9223690138862606
11
0.9169218632873253
12
0.9223308881518016
13
0.9108870229809949
14
0.9223690138862606
15
0.9207458199651356
16
0.924400790522645
17
0.927713132905387
18
0.9069627746345852
19
0.9231157378210836
20
0.9147921884916441
21
0.9145405986813814
22
0.928770487623671
23
0.9045391353850142
24
0.9226345442357636
25
0.9149052185794347
26
0.9233312942163608
27
0.9030385725625788
28
0.9148714859997246
29
0.9193748864159184
30
0.9164560279200761
31
0.9080835340946127
32
0.9174858277628622
33
0.9171499179721165
34
0.9145092694460673
35
0.9170942025890241
36
0.9014688269496115
37
0.9185033768355486
38
0.9171426992923174
39
0.9247235628941064
40
0.9220505869457986
41
0.9053930752960212
42
0.9214769482407945
43
0.9231157378210836
44
0.917090215583432
45
0.92079

### Adult

In [11]:
from data_loading import load_adult

In [12]:
(X_train, y_train, X_test, y_test), num_classes, n_num_feats, n_cat_feats, n_cat_vals = load_adult()
print("Train shape:", X_train[0].shape, X_train[1].shape) 
print("Test shape:", X_test[0].shape, X_test[1].shape)
print("Number of classes:", num_classes)

Train shape: (32561, 6) (32561, 8)
Test shape: (16281, 6) (16281, 8)
Number of classes: 2


In [13]:
(X_train, y_train, X_test, y_test), num_classes, n_num_feats, n_cat_feats, n_cat_vals = load_adult()


X_train_num, X_train_cat = X_train
X_test_num, X_test_cat = X_test


X_train_combined = np.hstack([X_train_num, X_train_cat])
X_test_combined  = np.hstack([X_test_num,  X_test_cat])


preprocessor = ColumnTransformer([
    ('num', StandardScaler(), list(range(n_num_feats))),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), 
     list(range(n_num_feats, n_num_feats + n_cat_feats)))
])
X_train_global_scaled = preprocessor.fit_transform(X_train_combined)
X_test_global_scaled  = preprocessor.transform(X_test_combined)


y_train_global = y_train
y_test_global  = y_test

In [14]:
import numpy as np
import joblib

# 1) load the pipeline
pipeline = joblib.load('/Users/abbaszal/Documents/global_acc_pipeline_fedfor.joblib')


In [15]:
# Prepare training DataFrame
t_feat = X_train_global_scaled.shape[1] - n_num_feats
feat_names = [f"num{i+1}" for i in range(n_num_feats)] + \
                [f"cat{i+1}" for i in range(t_feat)]
df_train = pd.DataFrame(X_train_global_scaled, columns=feat_names)
df_train["target"] = y_train_global

eps               = 1e-8
n_clients_list    = [10,20,30,40,50,60,70,80,90,100]
n_trials          = 100
base_random_seed  = 42
max_iters         = [100]
approach          = 'fedfor'
model_type = 'des'

# Saving directory
save_dir = "/Users/abbaszal/Documents/thimasfit11/adult_fedfor_all_joblib"
os.makedirs(save_dir, exist_ok=True)

for n_clients in n_clients_list:
    print(f"\n> n_clients = {n_clients}")

    # Dynamically get the corresponding payoff function
    #payoff_func_name = f"payoff_{n_clients}"
    #if payoff_func_name not in globals():
        #raise ValueError(f"Missing payoff function for {n_clients} clients")
    #payoff_func = globals()[payoff_func_name]


    for max_iter in max_iters:
        print(f"  max_iter = {max_iter}")

        # reset counters for this config
        counts_static = Counter()
        lottery_count = 0


        for trial in range(1, n_trials + 1):
            print(trial)

            rand_component = random.randint(0, 500)
            trial_seed = base_random_seed + trial + int(1000 * max_iter) + 2 * rand_component

            X_test_full_scaled = X_test_global_scaled.copy()
            y_test_full        = y_test_global.copy()

            subsample_fraction = 0.80

            X_test_glob, _, y_test_glob, _ = train_test_split(
                X_test_full_scaled, y_test_full,
                train_size=subsample_fraction,
                random_state=trial_seed,
                stratify=y_test_full
            )



            #from sklearn.model_selection import StratifiedKFold
            #skf = StratifiedKFold(n_splits=n_clients, shuffle=True, random_state=trial_seed)
            #parts = []
            #for _, client_idx in skf.split(df_train.drop(columns="target"), df_train["target"]):
                #df_c = df_train.iloc[client_idx].reset_index(drop=True)
                #parts.append(df_c)


            df_rem      = df_train.sample(frac=1, random_state=trial_seed).reset_index(drop=True)
            sample_size = len(df_train) // n_clients
            parts = []
            for idx in range(n_clients):
                if idx == n_clients - 1:
                    df_c = df_rem.copy()
                else:
                    sss = StratifiedShuffleSplit(n_splits=1, train_size=sample_size,
                                                    random_state=trial_seed)
                    tr_idx, _ = next(sss.split(df_rem, df_rem["target"]))
                    df_c      = df_rem.iloc[tr_idx]
                    df_rem    = df_rem.drop(df_c.index).reset_index(drop=True)
                parts.append(df_c.reset_index(drop=True))

            client_models     = []
            client_accs = {}
            for idx, df_c in enumerate(parts):
                X_c = df_c.drop(columns="target").to_numpy()
                y_c = df_c["target"].to_numpy()
                X_std = StandardScaler().fit_transform(X_c)

                if model_type.lower() == "logistic":
                    model = LogisticRegression(random_state=trial_seed,
                                                max_iter=max_iter)
                else:
                    model = DecisionTree(max_depth=max_iter,
                                                    random_state=trial_seed)

                model.fit(X_std, y_c)
                client_models.append(model)
                
    
                acc_global = accuracy_score(y_test_glob,
                                            model.predict(X_test_glob))
                client_accs[idx] = acc_global
            

            # coalition evaluation
            df_res = evaluate_coalitions2(
                client_models=client_models,
                client_global_accuracies=client_accs,
                n_clients=n_clients,
                aggregator_func=FederatedForest,
                X_test=X_test_glob,
                y_test=y_test_glob,
                corrupt_client_indices=[],
                approach=approach
            )

            # static‐game Nash counts
            df_ne = find_nash_equilibria_v2(df_res)
            if not df_ne.empty:
                for coalition in df_ne.index:
                    counts_static[coalition] += 1


            # lottery‐game incentive check
            vals      = np.array(list(client_accs.values()))
            mu_full   = vals.mean()
            sig_full  = vals.std(ddof=1)
            
            # 3) package into a 2-column array and predict
            X_new    = np.array([[mu_full, sig_full]])   # shape (1,2)
            payoff_f    = pipeline.predict(X_new)[0]
            print(payoff_f)


            has_incentive = any(
                acc > payoff_f + eps
                for acc in client_accs.values()
            )
            if not has_incentive:
                lottery_count += 1


        static_count = sum(counts_static.values())

        counts_df = pd.DataFrame([{
            'n_clients':           n_clients,
            'max_iter':            max_iter,
            'Static_Occurrences':  static_count,
            'Lottery_Occurrences': lottery_count
        }])
        fname = (f"Nash_Counts_{approach}"
                 f"_nclients_{n_clients}"
                 f"_maxiter_{max_iter}.csv")
        out_path = os.path.join(save_dir, fname)
        counts_df.to_csv(out_path, index=False)
        print(f"saved {fname}")



> n_clients = 10
  max_iter = 100
1
0.8639059426297929
2
0.8666515571876574
3
0.8676446100114524
4
0.8653541656998583
5
0.8656956143755072
6
0.867188118984061
7
0.8659429454765214
8
0.8664723644002528
9
0.8663338988038182
10
0.8658115742277566
11
0.8666322455889276
12
0.8654244703222185
13
0.8658563378267794
14
0.8665252660193412
15
0.8662104810918732
16
0.8678207425766719
17
0.8667225668099179
18
0.8670161778987288
19
0.8656513543849993
20
0.8664330940081559
21
0.8669880312764964
22
0.8659140046947943
23
0.8661149726152684
24
0.8647828676197398
25
0.8665375543410937
26
0.8673453470248551
27
0.8639105467997861
28
0.867192958310669
29
0.8638142919229101
30
0.8666515571876574
31
0.8668900524218911
32
0.8650715246433851
33
0.8653206048327174
34
0.8648374918611645
35
0.8655505514738557
36
0.8677274365843909
37
0.8650557919873818
38
0.8641623103776059
39
0.8671514970652713
40
0.8674191835045049
41
0.8661279043276595
42
0.8669749350490988
43
0.8657267585598317
44
0.8666496348588333
45
0.865